In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Implementing a object recognition and tracking algorithm 

1. Use YOLO -> generate bounding box tags [This uses pre-trained weights, so eventually might need retraining]
2. Given bouding box data from the algorithm, instantiate a tracker by calling the sort algorithm method, which tracks and collapses trackers frame to frame based on YOLO output data.
3. Save tracker outputs in text format specified by MOT205 
4. From the MOT2015 data set, run python implementation of metric calculator to evaluate performace for differnt test sets 

TO DO:
1. YOLO needs fixes -> slightly buggy right now, but generates bounding boxes fine
2. Replace tracker (SORT/KALMAN) with RNN based tracker (on-going)
3. Revaluate benchmark performance after 1&2 


Dependencie
filterpy, cv2, numpy, torch, 

In [0]:
import cv2
import filterpy
from sort import *
from models import *
from utils_ import *

import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

In [0]:
# Configre YOLO weights and classes 

config_path='/content/drive/My Drive/Colab Notebooks/object_tracking/pytorch_objectdetecttrack/config/yolov3.cfg'
weights_path='/content/drive/My Drive/Colab Notebooks/object_tracking/pytorch_objectdetecttrack/yolov3.weights'
class_path='/content/drive/My Drive/Colab Notebooks/object_tracking/pytorch_objectdetecttrack/config/coco.names'
img_size=416
conf_thres=0.8
nms_thres=0.4

# Load model and weights
# defines a YOLO instantiation using the Darknet flow (found easily on Github)

model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.cuda()
model.eval()
classes = load_classes(class_path)
Tensor = torch.cuda.FloatTensor

In [0]:
# Define a track image function to extract box data for a single image
# We will use this again and again to do frame by frame object detection 

##### Detect Image Function #################
def detect_image(img):
    # scale and pad image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])
    # convert image to Tensor
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))
    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(input_img)
        detections = non_max_suppression(detections, 80, conf_thres, nms_thres)
    return detections[0]

In [0]:
# SORT + YOLO 
# Set input data (video) path & output (video) path 

# in our case , we ran this after mounting on my collab acoount 

videopath = '/content/drive/My Drive/Colab Notebooks/object_tracking/pytorch_objectdetecttrack/Videos/Venice-2.mp4'
savepath = '/content/drive/My Drive/Colab Notebooks/object_tracking/pytorch_objectdetecttrack'


%pylab inline 
from IPython.display import clear_output

cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]



# initialize Sort object and video capture
from sort import *
vid = cv2.VideoCapture(videopath)
mot_tracker = Sort() 



# Section to create a cv2 instance to handle video input and process it frame by 
# frame 

fourcc = cv2.VideoWriter_fourcc(*'XVID')
ret,frame=vid.read()
vw = frame.shape[1]
vh = frame.shape[0]
print ("Video size", vw,vh)
outvideo = cv2.VideoWriter(videopath.replace(".mp4", "-det.mp4"),fourcc,20.0,(vw,vh))


frames = 0
starttime = time.time()
obj_id = 0 

with open('%s/output/det.txt'%(savepath),'w') as out_file:
  for ii in range(599):
      frames += 1
      ret, frame = vid.read()
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      pilimg = Image.fromarray(frame)
      detections = detect_image(pilimg)

      img = np.array(pilimg)
      pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
      pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
      unpad_h = img_size - pad_y
      unpad_w = img_size - pad_x

      if detections is not None:  
          tracked_objects = mot_tracker.update(detections.cpu())
          unique_labels = detections[:, -1].cpu().unique()
          n_cls_preds = len(unique_labels)
          for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
              if (int(cls_pred==0)):
                box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
                box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
                y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
                x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])

                color = colors[int(obj_id) % len(colors)]
                color = [i * 255 for i in color]
                cls = classes[int(cls_pred)]
                cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h), color, 4)
                cv2.rectangle(frame, (x1, y1-35), (x1+len(cls)*19+60, y1), color, -1)
                #cv2.putText(frame, cls + "-" + str(int(obj_id)), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
                cv2.putText(frame, cls, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
  
      #fig=figure(figsize=(12, 8))
      #title("Video Stream")
      #imshow(frame)
      #show()
      #clear_output(wait=True)
      outvideo.write(frame)
      
      for d in tracked_objects: 
        if(int(cls_pred==0)):
          print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1'%(ii,d[4],d[0],d[1],d[2]-d[0],d[3]-d[1]),file=out_file) 
            
      ch = 0xFF &cv2.waitKey(1)
      if ch == 27:
        break
   
  
totaltime = time.time()-starttime
print(frames, "frames", totaltime/frames, "s/frame")
#cv2.destroyAllWindows()
outvideo.release()




